In [4]:
from sotodlib import core
import numpy as np
import so3g
from scipy.optimize import curve_fit
import ephem, pickle
from datetime import datetime
from so3g.proj import coords, quat
from astropy.table import Table


In [15]:
#2D Gaussian Beam model
def gaussian2d(xieta, a, xi0, eta0, fwhm_xi, fwhm_eta, phi):
    xi, eta = xieta
    xi_rot = xi*np.cos(phi) - eta*np.sin(phi)
    eta_rot = xi*np.sin(phi) + eta*np.cos(phi)
    factor = 2*np.sqrt(2*np.log(2))
    xi_coef = -0.5 * (xi_rot-xi0)**2/(fwhm_xi/factor)**2
    eta_coef = -0.5 * (eta_rot-eta0)**2/(fwhm_eta/factor)**2
    return(a*np.exp(xi_coef+eta_coef))


In [16]:
print("Starting SAT Beams")
name = 'fitting_det0064_90_test1'
path = '/home/jorlo/dev/gaussian_beams/'
sat_beam = pickle.load(open(path + name + '.pkl', 'rb'), encoding = 'bytes')

data = sat_beam['data']

X, Y = sat_beam['mesh'][0], sat_beam['mesh'][1]

Starting SAT Beams


In [17]:
######Parameter setting area######
radius_cut = 5. #arcmin, cut-out radius for beam fitting
p0 = [8.03922740e+02, -7.93350161e+00, -1.11729140e+01, 1.99074908e-01, 1.95726535e-01, np.pi/6] #initial parameter values, organized as
                                            #amp, xi0, eta0, fwhm_xi, fwhm_eta, phi
offset_bound = np.radians(5./60) # bound for the offset fit at 5 arcmin
fwhm_bound = 5e-4 # radians, upper bound for the fwhm fit
bounds = ((-np.inf, -offset_bound, -offset_bound, 0., 0., 0),
          (np.inf, offset_bound, offset_bound, fwhm_bound, fwhm_bound, 2*np.pi),)


In [28]:
x = np.linspace(np.amin(X), np.amax(X), 1001)
y = np.linspace(np.amin(Y), np.amax(Y), 1001)
xieta = np.vstack((x, y))
test = [-7.93350161e+00,-1.11729140e+01]
print(gaussian2d(test, p0[0], p0[1],p0[2],p0[3], p0[4], p0[5]))
popt, _ = curve_fit(gaussian2d, xieta, data, p0=p0, bounds=bounds,)


0.0


ValueError: `fun` must return at most 1-d array_like.